<a href="https://colab.research.google.com/github/Coduck-repo/fun-fun-stock/blob/master/NaverStock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

# 종목코드 가져오기
stock_code = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

# 보합 https://finance.naver.com/sise/sise_steady.nhn?sosok=1
# 상승 https://finance.naver.com/sise/sise_rise.nhn?sosok=1
kospi_url = "https://finance.naver.com/sise/sise_steady.nhn?sosok=0"
kosdaq_url = "https://finance.naver.com/sise/sise_steady.nhn?sosok=1"


def GetSteadyStock(type):
    if (type == 'kospi'):
        url = kospi_url
    else:
        url = kosdaq_url

    html = urlopen(url)
    content = html.read().decode('euc-kr', 'replace').encode('utf-8', 'replace')
    soup = BeautifulSoup(content, 'html.parser')
    table = soup.find("table", class_="type_2")
    sise = []
    for tr in table.find_all("tr")[1:]:
        tds = tr.find_all("td")
        stock = []
        for td in tds[1:10]:
            value = td.get_text().strip().replace(',', '')
            try:
                number = int(value)  # 수치 변환
                stock.append(number)
            except ValueError:  # 문자열
                stock.append(value)
        if len(stock):
            sise.append(tuple(stock))

    df = pd.DataFrame(sise, columns=['종목명', '현재가', '전일비', '등락률', '거래량', '매수호가', '매도호가', '매수총잔량', '매도총잔량'])
    steady_stock = pd.merge(df, stock_code.rename(columns= {'회사명':'종목명'})[['종목명','종목코드']])
    return steady_stock

kospi_df = GetSteadyStock('kospi')
kosdaq_df = GetSteadyStock('kosdaq')

# 정렬 (거래량)
#df1 = df1.sort_values(by='거래량', ascending=False)

# 종목토론실 게시물 컨텐츠
# 페이지 크롤링 필요함
def StockBoardContent(code):
    url = "https://finance.naver.com/item/board.nhn?code=%06d" % code

    board = []
    html = urlopen(url)
    content = html.read().decode('euc-kr', 'replace').encode('utf-8', 'replace')
    soup = BeautifulSoup(content, 'html.parser')
    table = soup.find("table", class_="type2")
    for tr in table.find_all("tr")[1:]:
        item = []
        tds = tr.find_all("td")
        for td in tds:
            value = td.get_text().strip().replace('\t','').replace('\t','').replace('\n','')
            item.append(value)
        
        if len(item) != 1:
          board.append(tuple(item))

    df = pd.DataFrame(board, columns=['날짜','제목','투자의견','글쓴이','조회','공감','비공감'])
    return df

# 전 종목 게시판 크롤링
#for index, row in kospi_df.iterrows():
#    df = StockBoardCount(row['종목코드'])

In [0]:
df = StockBoardContent(990)

In [32]:
df.sort_values(by='날짜', ascending=False)

,날짜,제목,투자의견,글쓴이,조회,공감,비공감
0,2019.12.17 19:10,주식,의견없음,dnsk****,58,1,1
2,2019.12.17 18:58,잡상인임~클릭금지,의견없음,kgbs****,90,1,0
1,2019.12.17 18:54,수익 내신 분들 축하드립니다,의견없음,hand****,208,3,2
3,2019.12.17 18:45,이런주식에 기사 하나안쓰는 기관투자자들 ...,의견없음,a880****,290,3,1
4,2019.12.17 18:33,잘먹고 갑니다.,의견없음,ctac****,340,4,0
15,2019.12.17 18:32,낼은 상[1],의견없음,9999****,90,1,0
5,2019.12.17 18:27,알려주세요![1],의견없음,doll****,404,0,3
6,2019.12.17 18:25,db하이텍...올 해의 승리자는?[2],의견없음,docj****,444,3,3
7,2019.12.17 18:11,좀 오르니 안티들이 와서 짖는구나,의견없음,ds81****,305,2,3
8,2019.12.17 18:01,"●아래 의견 글,,,,,, 타당함.[1]",의견없음,pyun****,537,0,3
